In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
crash=pd.read_csv('../input/nys-motor-vehicle-crashes-and-insurance-reduction/motor-vehicle-crashes-case-information-three-year-window.csv')
vehicle=pd.read_csv('../input/nys-motor-vehicle-crashes-and-insurance-reduction/motor-vehicle-crashes-vehicle-information-three-year-window.csv')
person=pd.read_csv('../input/nys-motor-vehicle-crashes-and-insurance-reduction/motor-vehicle-crashes-individual-information-three-year-window.csv')

In [ ]:
vehper=vehicle.merge(person,left_on='Case Vehicle ID',right_on='Case Vehicle ID')

In [ ]:
vehper[:10].T

In [ ]:
crash[:10].T

In [ ]:
vehicle[:10].T

In [ ]:
from sklearn.preprocessing import LabelEncoder
if True:   #labelencode instead of using average is better
    for c in vehper.columns:
        if vehper[c].dtype == 'object':
            lbl = LabelEncoder()
            lbl.fit(list(vehper[c].values)) 
            vehper[c] = lbl.transform(list(vehper[c].values))


In [ ]:
vehper[:10].T

In [ ]:
def clustertechniques(dtrain,label,indexv,dtest,lengte):
    # One Hot Encode target mean()
    cols=[ci for ci in dtrain.columns if ci not in [indexv,'index',label]]
    coltype=dtrain.dtypes
    for ci in cols:
        if (coltype[ci]=="object"):
            codes=dtrain[[ci,label]].groupby(ci).mean().sort_values(label)
            codesdict=codes[label].to_dict()
            dtrain[ci]=dtrain[ci].map(codesdict)
            if len(dtest)>0:
                dtest[ci]=dtest[ci].map(codesdict)
    print(dtest)
    #split data or use splitted data
    if len(dtest)==0:
        dtrain = dtrain[:lengte]
        print(dtrain.info())
        from sklearn.model_selection import train_test_split
        X_train,X_test,Y_train,Y_test = train_test_split(dtrain.drop(label,axis=1),dtrain[label],test_size=0.25,random_state=0)
        print('train - test',X_train.shape,X_test.shape)
    else:

        X_test=dtest.dropna(axis=1).drop(indexv,axis=1)
        Y_test=dtest[indexv]
        X_train=dtrain[X_test.columns]
        Y_train=dtrain[label]        
        print('train - test',X_train.shape,X_test.shape)
    lenxtr=len(X_train)


    import matplotlib.pyplot as plt 
    from sklearn import preprocessing
    scale = preprocessing.MinMaxScaler().fit(X_train)
    X_train = scale.transform(X_train)
    X_test = scale.transform(X_test)

        
    from sklearn.decomposition import PCA,TruncatedSVD,NMF,FastICA
    from umap import UMAP  # knn lookalike of tSNE but faster, so scales up
    from sklearn.manifold import TSNE #limit number of records to 100000

    clusters = [Dummy(1),
                PCA(n_components=0.98,random_state=0,whiten=True),
                #FastICA(n_components=7,random_state=0),
                TruncatedSVD(n_components=10, n_iter=7, random_state=42),
                #NMF(n_components=10,random_state=0),            
                #UMAP(n_neighbors=5,n_components=10, min_dist=0.3,metric='minkowski'),
                #TSNE(n_components=2,random_state=0)
                ] 
    clunaam=["raw",'PCA','tSVD']#,'ICA','tSVD','nmf','UMAP','tSNE']
    
    
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC, LinearSVC,NuSVC
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
    from sklearn.neural_network import MLPClassifier,MLPRegressor
    from sklearn.linear_model import PassiveAggressiveClassifier,Perceptron,SGDClassifier,LogisticRegression
    import xgboost as xgb
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
    
    classifiers = [LogisticRegression( solver="lbfgs",max_iter=500,n_jobs=-1),
                   PassiveAggressiveClassifier(max_iter=50, tol=1e-3,n_jobs=-1),    
                   KNeighborsClassifier(n_neighbors=5,n_jobs=-1),
                   RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1, oob_score=True),
                   ExtraTreesClassifier(n_estimators=10, max_depth=50, min_samples_split=5, min_samples_leaf=1, random_state=None, min_impurity_decrease=1e-7),
                   #SVC(gamma='auto'),                   
                   MLPClassifier(alpha=0.510,activation='logistic'),
                   SGDClassifier(average=True,max_iter=100),
                   xgb.XGBClassifier(n_estimators=50, max_depth = 9, learning_rate=0.01, subsample=0.75, random_state=11),
                    #SVC(kernel="rbf", C=0.025, probability=True),
                    #NuSVC(probability=True),
                    DecisionTreeClassifier(),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
                    GaussianNB(),
                    LinearDiscriminantAnalysis(),
                    QuadraticDiscriminantAnalysis()
                  ]
    clanaam= ['Logi','passiv','KNN','rFor','Xtr','MLP','SGD','xgb','Decis','Ada','GradB','GausNB','LinDis','QuadDis']#['Logi','KNN','rFor','SVC','MLP','SGD']
    from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
    
    results=[]


    #cluster data
    for clu in clusters:
        clunm=clunaam[clusters.index(clu)] #find naam
        X_total_clu = clu.fit_transform(np.concatenate( (X_train,X_test),axis=0))
        print(X_total_clu.shape)
        plt.scatter(X_total_clu[:lenxtr,0],X_total_clu[:lenxtr,1],c=Y_train.values,cmap='prism')
        plt.title(clu)
        plt.show()
        
        #classifiy 
        for cla in classifiers:
            import datetime
            start = datetime.datetime.now()
            clanm=clanaam[classifiers.index(cla)] #find naam
            
            print('    ',cla)
            #cla.fit(X_total_clu,np.concatenate( (Y_train,Y_test)) )
            cla.fit(X_total_clu[:lenxtr],Y_train )
            
            #predict
            trainpredi=cla.predict(X_total_clu[:lenxtr])
            print(classification_report(trainpredi,Y_train))            
            testpredi=cla.predict(X_total_clu[lenxtr:])            

            #testpredi=converging(pd.DataFrame(X_train),pd.DataFrame(X_test),Y_train,pd.DataFrame(testpredi),Y_test,clu,cla) #PCA(n_components=10,random_state=0,whiten=True),MLPClassifier(alpha=0.510,activation='logistic'))
            
            if len(dtest)==0:
                test_score=cla.score(X_total_clu[lenxtr:],Y_test)
                accscore=accuracy_score(testpredi,Y_test)
                li = [clunm,clanm,test_score,accscore]
                results.append(li)                
                print(confusion_matrix(testpredi,Y_test))

                plt.title(clanm+'test accuracy versus unknown:'+np.str(test_score)+' '+np.str(accscore)+' and test confusionmatrix')
                plt.scatter(x=Y_test, y=testpredi, marker='.', alpha=1)
                plt.scatter(x=[np.mean(Y_test)], y=[np.mean(testpredi)], marker='o', color='red')
                plt.xlabel('Real test'); plt.ylabel('Pred. test')
                plt.show()


            else:
                testpredlabel=le.inverse_transform(testpredi)  #use if you labellezid the classes 
#                testpredlabel=testpredi

                submit = pd.DataFrame({indexv: dtest[indexv],label: testpredlabel})
                filenaam='subm_'+clunm+'_'+clanm+'.csv'
                submit.to_csv(path_or_buf =filenaam, index=False)
                
            print(clanm,'0 classifier time',datetime.datetime.now()-start)
            
    if len(dtest)==0:       
        print(pd.DataFrame(results).sort_values(3))
        submit=[]
    return submit

#Custom Transformer that extracts columns passed as argument to its constructor 
class Dummy( ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def fit_transform( self, X, y = None ):
        return X 

#clustertechniques(train[significant_features],'target','id',[],10000)
clustertechniques(vehper.drop(['Case Vehicle ID','Partial VIN'],axis=1).fillna(0),'Injury Severity','Case Vehicle ID',[],100000)